In [1]:
import matplotlib.pyplot as plt
from classes.backtest_bloom import Backtester, Config, Frequency, TypeOptiWeights
import blpapi
import numpy as np
import pandas as pd
import datetime as dt
from classes.module import BLP
from classes.strategies_bloom import Strategies, TypeStrategy

C:\Users\nicol\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\nicol\Videos\Bloomberg\Bloomberg\classes\backtest_bloom.py:16: UserWarning: Running this module requires the package: polars 0.15.14
  warn('Running this module requires the package: polars 0.15.14')


In [ ]:
type_strat = TypeStrategy.momentum.value
DATE = blpapi.Name("date")
ERROR_INFO = blpapi.Name("errorInfo")
EVENT_TIME = blpapi.Name("EVENT_TIME")
FIELD_DATA = blpapi.Name("fieldData")
FIELD_EXCEPTIONS = blpapi.Name("fieldExceptions")
FIELD_ID = blpapi.Name("fieldId")
SECURITY = blpapi.Name("security")
SECURITY_DATA = blpapi.Name("securityData")

blp = BLP(DATE, SECURITY, SECURITY_DATA, FIELD_DATA)
strFields = ["PX_LAST", "INDX_MWEIGHT_HIST"]
tickers = ["CAC Index"]
startDate = dt.datetime(2020, 1, 2)
endDate = dt.datetime(2023, 2, 10)

In [ ]:
dictCompoIndex = blp.compo_per_date_old(strSecurity=tickers, strFields=strFields,
                                        strOverrideField="END_DATE_OVERRIDE", strOverrideValue=startDate,
                                        strEndDate=endDate)
idx_df = pd.DataFrame(dictCompoIndex).T
unique_idx = sorted(list(set(idx_df.sum()[0])))
list_index = list(map(lambda x: x + " Equity", unique_idx))

dictTickersTime = {k: list(map(lambda x: x + " Equity", np.array(v).flatten().tolist())) for k, v in
               dictCompoIndex.items()}
dfHistory = blp.bdh(list_index, ['PX_LAST'], startDate, endDate)  # CUR_MKT_CAP # TOTAL_EQUITY # PX_LAST
if type_strat == TypeStrategy.btm.value:
    dfHistory_btm = blp.bdh(list_index, ['CUR_MKT_CAP'], startDate, endDate)
else:
    dfHistory_btm = None

In [ ]:
configuration = Config(universe=list_index,
                       start_ts=startDate,
                       end_ts=endDate,
                       strategy_code=type_strat,
                       name_index=tickers,
                       frequency=Frequency.DAILY)

backtest = Backtester(config=configuration, data=dfHistory,
                      compo=dictTickersTime,
                      intReshuffle=10,
                      lag1=30, lag2=200, boolGeneric=False, strat=TypeOptiWeights.MIN_VARIANCE,
                      other_data=dfHistory_btm)

In [ ]:
back = backtest.compute_levels()
backtest_res = pd.DataFrame(back)
plt.figure(1)
plt.plot(backtest_res['ts'], backtest_res['close'])
plt.xlabel('date')
plt.ylabel('ptf level')
plt.title('ptf evolution - momentum')
plt.legend()